In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from scipy.signal import savgol_filter
from scipy import sparse
from scipy.sparse.linalg import spsolve
from joblib import Parallel,delayed
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
import itertools
from sklearn.metrics import plot_confusion_matrix

def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1 - p) * (y < z)
    return z

def plot_confusion_matrix(cm, classes,
                          normalize=False,fontsize=18,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = cm * 100
        
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    fig, ax = plt.subplots(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=15, fontname = "Helvetica",rotation=45)
    plt.yticks(tick_marks, classes, fontsize=15, fontname = "Helvetica")
    plt.tick_params(direction='in', length=8, width=0, colors='black')
    fmt = '.1f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

fnames=sorted(glob('Data_Pathway/*'))
print(fnames)
files=[np.loadtxt(f) for f in fnames]
map=[f[:,3] for f in files]
map=[np.reshape(m,(-1,1011)) for m in map]
map_full = np.concatenate([m[:,:] for m in map],axis=0)
y=np.concatenate([np.repeat(i, len(m)) for i, m in enumerate(map)], axis=0)
map_full= savgol_filter(map_full, 11, 3, axis=0)
back = Parallel(n_jobs=8)(delayed(baseline_als)(j, 100000, 0.001) for j in map_full)
map_full= np.subtract(map_full, back)
map_full= np.reshape(map_full, (-1, 1011))
#X= minmax_scale(map_full, axis=1)
divide=map_full[:,999:1000]
X=map_full/divide

x_train, x_test, y_train, y_test=train_test_split(X,y,test_size=0.2,stratify=y, random_state=42,shuffle=True)

skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
predicted_targets = np.array([])
actual_targets = np.array([])

fold_var = 1

for train_index, validation_index in skf.split(x_train,y_train): 
    # keeping validation set apart and oversampling in each iteration using smote 
    X_tra, X_val = x_train[train_index],x_train[validation_index]
    y_tra, y_val = y_train[train_index],y_train[validation_index]
    
    pca = PCA(n_components=22)
    pca.fit(X_tra)
    
    X_tra= pca.transform(X_tra)
    X_val= pca.transform(X_val)
    
    sm = SMOTE()
    X_train_res, y_train_res = sm.fit_resample(X_tra, y_tra)
    print (X_train_res.shape, y_train_res.shape)
    
    # training the model on oversampled 9 folds of training set 
    svc = SVC()
    svc.fit(X_train_res, y_train_res)
    # testing on 1 fold of validation set
    y_preds = svc.predict(X_val)

    
    predicted_targets = np.append(predicted_targets, y_preds)
    actual_targets = np.append(actual_targets, y_val)
    fold_var += 1


#'0'=control class, '1'=5 fM, '2'=50 fM, '3'=0.5 pM,'4'=5 pM,'5'=50 pM,'6'=0.5 nM,'7'=5 nM, '8'=50 nM, '9'=0.5uM, 
#'10'=5 uM,'11'=50 uM,'12'=0.5 mM,'13'=5 mM
class_names=['0', '1', '2', '3', '4', '5','6','7','8','9','10','11','12','13']
cn_matrix = confusion_matrix(actual_targets,predicted_targets)
plot_confusion_matrix(cn_matrix, classes=class_names, normalize=True)